# 🛡️ Lab 1: Prompt Engineering for Security Tasks

## 🧪 Task: Extract CVEs from a Noisy Security Log

In this lab, you'll practice crafting prompts for LLMs to extract structured vulnerability data (CVEs) from unstructured system logs.
This simulates how security teams might automate incident summarization or threat detection workflows using GenAI.


## 🛠️ Setup Instructions

Before you begin, ensure you:
- Have an OpenAI API key and environment variable (`openai.api_key`)
- Are using GPT-3.5-Turbo or GPT-4
- Are familiar with basic Python

In this task, you'll analyze a noisy system log containing multiple CVEs and generate a structured summary using the LLM.

The logs may contain real CVE patterns (e.g., CVE-YYYY-NNNN), which you need to extract using a well-engineered prompt.


## 1. Extract CVEs from a noisy log

In [ ]:
import openai

from google.colab import userdata
# Insert your OpenAI API key here
api_key = userdata.get('openai.api_key')

# ✅ Create the OpenAI client
client = openai.OpenAI(api_key=api_key)

# Sample Noisy input
noisy_log = """
System Alert: Vulnerability ID CVE-2023-12345 detected on host srv-12.
User admin logged in via SSH. Possible exploit associated with CVE-2022-9999.
Refer to advisory: https://nvd.nist.gov/vuln/detail/CVE-2023-12345
"""



####Vague Prompt

In [ ]:
prompt = f"Can you get CVEs from this text: {noisy_log}"
# ChatCompletion using new SDK style
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0.2
)

# Print result
output_text = response.choices[0].message.content
print("🔎 LLM Output:\n", output_text)


🔎 LLM Output:
 Yes, the CVEs mentioned in the text are CVE-2023-12345 and CVE-2022-9999.


### Clean prompt



In this example, we refine the prompt to help the LLM focus on just the CVE extraction.


In [ ]:

# Prompt
prompt = f"""
You are a security analyst assistant.
Extract all CVE identifiers from the following text and return them as JSON.

Text:
'''{noisy_log}'''

Return in this format:
{{"cves": ["CVE-XXXX-YYYY", ...]}}
"""

# ChatCompletion using new SDK style
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0.2
)

# Print result
output_text = response.choices[0].message.content
print("🔎 LLM Output:\n", output_text)

🔎 LLM Output:
 {
  "cves": ["CVE-2023-12345", "CVE-2022-9999"]
}


## 🧠 Why This Matters

Large Language Models are highly sensitive to the way you structure prompts.  
In security use cases — like extracting CVEs from messy logs — it's important to:

- Minimize ambiguity
- Provide format expectations
- Isolate the actual task from the noise

## 2. Secure Code Review with GenAI




In [ ]:
# Insecure code to review
code_snippet = """
def authenticate(user, password):
    if password == "admin123":
        return True
    return False
"""

# Prompt for secure code analysis
prompt = f"""
You are a secure code reviewer.

Analyze the following Python function for security issues and respond with:
1. Identified vulnerabilities
2. Mapped OWASP Top 10 category (2021)
3. Estimated CVSS score (if applicable)
4. Recommendations for fixing
5. A corrected version of the code

Code:
'''{code_snippet}'''

Respond in this JSON format:
{{
  "vulnerabilities": ["..."],
  "owasp_mapping": ["..."],
  "cvss_estimate": "X.X",
  "recommendations": "...",
  "fixed_code": \"\"\"...\"\"\"
}}
"""

# Call OpenAI Chat API
response = client.chat.completions.create(
    model="gpt-4",  # or "gpt-3.5-turbo"
    messages=[{"role": "user", "content": prompt}],
    temperature=0.2
)

# Output the result
print(response.choices[0].message.content)

{
  "vulnerabilities": ["Hard-coded credentials"],
  "owasp_mapping": ["A3:2021-Injection"],
  "cvss_estimate": "9.8",
  "recommendations": "Never hard-code credentials in your code. Instead, use secure methods for storing and retrieving credentials. Implement a proper authentication system that securely stores and checks user credentials. Hash and salt passwords for storage. Use a library or framework that has been designed to provide secure authentication features.",
  "fixed_code": """
def authenticate(user, password):
    # This is a placeholder for the actual implementation
    # In a real-world application, you would use a secure method to store and check passwords
    # For example, you might store a hash of the password and compare it to a hash of the input
    stored_password_hash = get_password_hash_for_user(user)
    input_password_hash = hash_password(password)
    if stored_password_hash == input_password_hash:
        return True
    return False
"""
}


## 📋 Review the Results

Now that the LLM has provided a secure code analysis, let’s:

- Clean and parse the JSON response
- Fix formatting issues (like escaped characters in the returned code)
- Display the results as a readable table

This makes it easier to verify what the model found, how it mapped to OWASP Top 10, and whether the recommendations make sense.

> 💡 This step simulates how GenAI results could be integrated into code review dashboards or CI/CD pipelines.


In [ ]:
import json
import re
import pandas as pd

def clean_openai_json(raw_output):
    # Step 1: Extract the JSON object using regex
    match = re.search(r"\{.*\}", raw_output, re.DOTALL)
    if not match:
        raise ValueError("No JSON object found in response.")

    json_text = match.group()

    # Step 2: Replace triple-quoted code block with escaped string
    json_text = re.sub(
        r'\"\"\"(.*?)\"\"\"',
        lambda m: json.dumps(m.group(1).strip()),
        json_text,
        flags=re.DOTALL
    )

    return json.loads(json_text)

# Example usage
try:
    parsed_output = clean_openai_json(response.choices[0].message.content)

    # ✅ Fix newlines in code
    parsed_output["fixed_code"] = parsed_output["fixed_code"].encode().decode("unicode_escape")

    # ✅ Convert to a clean DataFrame
    df = pd.DataFrame([{
        "vulnerabilities": "; ".join(parsed_output.get("vulnerabilities", [])),
        "owasp_mapping": "; ".join(parsed_output.get("owasp_mapping", [])),
        "cvss_estimate": parsed_output.get("cvss_estimate", ""),
        "recommendations": parsed_output.get("recommendations", ""),
        "fixed_code": parsed_output.get("fixed_code", "")
    }])

    # Print as DataFrame (can also use display(df) in notebooks)
    print("\n✅ Full Secure Code Review Table:\n")
    print(df.to_markdown(index=False))

except Exception as e:
    print("❌ Failed to clean/parse:", e)



✅ Full Secure Code Review Table:

| vulnerabilities        | owasp_mapping     |   cvss_estimate | recommendations                                                                                                                                                                                                                                                                                                                            | fixed_code                                                                                      |
|:-----------------------|:------------------|----------------:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------

### 3. PII Redaction

#### **SpaCy / Regex**

## 🛡️ PII Redaction Using spaCy + Regex

In this section, you’ll build a lightweight PII redaction tool using `spaCy` for Named Entity Recognition (NER) and regular expressions for custom pattern matching (e.g., emails, phone numbers, addresses).

spaCy is a free open-source library for Natural Language Processing in Python. It features NER, POS tagging, dependency parsing, word vectors and more.

---

### 🔍 Why Use spaCy for PII Detection?

✅ **Pre-trained NER:** Detects entities like names, organizations, dates, and locations  
✅ **Regex Flexibility:** Easily extend redaction to custom patterns (SSNs, IPs, etc.)  
✅ **Lightweight & Fast:** Ideal for real-time or local processing  
✅ **No LLM required:** Works without external APIs

---

### 🔧 Installation

Run the following to install spaCy and download the English language model:

```bash
pip install spacy --quiet
python -m spacy download en_core_web_sm


📚 References

[spaCy Official Docs](https://www.google.com/url?q=https%3A%2F%2Fspacy.io%2F)

[NER Overview](https://spacy.io/usage/linguistic-features#named-entities)

[spaCy API Reference](https://spacy.io/api)

In [ ]:
!pip install spacy --quiet
!python -m spacy download en_core_web_sm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 56.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import re

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define a function to redact PII
def redact_pii(text):
    doc = nlp(text)
    redacted_text = text

    # Named Entity Recognition (NER)-based redaction
    for ent in doc.ents:
        if ent.label_ in ["PERSON", "GPE", "ORG", "LOC", "DATE", "EMAIL"]:
            redacted_text = redacted_text.replace(ent.text, f"[REDACTED-{ent.label_}]")

    # Regex-based redaction
    patterns = {
        "PHONE": r"\b\d{3}[-.\s]??\d{3}[-.\s]??\d{4}\b",
        "EMAIL": r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",
        "ADDRESS": r"\d{1,5}\s\w+\s\w+\.?",  # rough pattern
    }

    for label, pattern in patterns.items():
        redacted_text = re.sub(pattern, f"[REDACTED-{label}]", redacted_text)

    return redacted_text


In [ ]:
sample_text = """
Hi, my name is Alice Johnson. You can email me at alice.j@example.com or call me at 555-123-4567.
I live at 42 Elm Street, Springfield. I work for OpenAI.
"""

safe_text = redact_pii(sample_text)

print("✅ Redacted Text:\n")
print(safe_text)


✅ Redacted Text:


Hi, my name is [REDACTED-PERSON]. You can email me at [REDACTED-ORG] or call me at [REDACTED-PHONE].
I live at [REDACTED-ADDRESS], [REDACTED-GPE]. I work for [REDACTED-PERSON].



#### **DSPy Framework**

## 🤖 PII Redaction Using DSPy (LLM + Chain of Thought)

Now let’s explore how to use **DSPy**, a structured prompting framework from Stanford, to redact PII using large language models. “DSPy is a framework for programming LLMs declaratively — meaning, instead of crafting one perfect prompt, you define modules and constraints, then let DSPy optimize them automatically with your examples.”

Instead of rule-based patterns, DSPy lets you:
- Define **input/output schemas** for clean and consistent behavior
- Use **Chain of Thought reasoning** to break down and explain each step
- Build reproducible, testable GenAI tools for redaction, extraction, and more

---

### 💡 Why Use DSPy for PII Tasks?

✅ **Schema-based prompting:** Define what the model should take in and return  
✅ **LLM-first design:** Easy integration with OpenAI, Claude, Mistral, etc.  
✅ **Chain of Thought support:** Helps with explainability and traceability  
✅ **Ideal for security + compliance workflows:** Enforce structured outputs

---

### 🔧 Installation

To install DSPy, run:

```bash
pip install -U dspy-ai --quiet


References:
*   https://dspy.ai/
*   https://dspy.ai/community/use-cases/

In [ ]:
!pip install -U dspy-ai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 16.4 MB/s eta 0:00:00


In [ ]:
import dspy
import os

# Securely load your API key
os.environ["OPENAI_API_KEY"] = userdata.get('openai.api_key')

# Initialize the LLM interface using the proper API
lm = dspy.LM("openai/gpt-3.5-turbo", api_key=os.environ["OPENAI_API_KEY"])

# Configure DSPy to use this LLM
dspy.settings.configure(lm=lm, temperature=0.2)


In [ ]:
class PiiRedactionSig(dspy.Signature):
    text = dspy.InputField(desc="Raw input text with possible PII (Name, Email, Phone, SSN, etc.)")
    redacted_text = dspy.OutputField(desc="The text with PII redacted using [REDACTED] tags")
    pii_entities = dspy.OutputField(desc="List of extracted PII values and their types in JSON")


In [ ]:
redactor = dspy.ChainOfThought(PiiRedactionSig)
result = redactor(text="My name is Alice. Email: alice@example.com. SSN: 123-45-6789.")

print("🔒 Redacted:\n", result.redacted_text)
print("📋 Entities:\n", result.pii_entities)


🔒 Redacted:
 My name is [REDACTED]. Email: [REDACTED]. SSN: [REDACTED].
📋 Entities:
 {"Name": "Alice", "Email": "alice@example.com", "SSN": "123-45-6789"}


In [ ]:
#result.rationale

In [ ]:
test_texts = [
    "My name is Alice. Email: alice@example.com. SSN: 123-45-6789.",
    "Contact John via john.doe@gmail.com. He lives at 123 Main St.",
    "Sarah's phone number is 555-123-4567 and her address is 55 Sunset Blvd.",
    "James works at Google. His email is james@google.com.",
    "Emma lives in San Francisco, California. Her number is 415-555-7890.",
    "Liam's passport number is X12345678.",
    "Olivia’s home is 98 Hill Rd, Boston. Reach her at olivia.b@example.com.",
    "William was born on 01/15/1990 and his SSN is 321-54-9876.",
    "Ava’s insurance policy number is INSU123456789.",
    "Mason’s IP address is 192.168.1.100.",
    "Isabella has a crypto wallet: 0x5FbDB2315678afecb367f032d93F642f64180aa3",
    "Ethan submitted his driver's license: D12345678.",
    "Mia registered from device ID: DEV-77889900.",
    "Sophia’s card number is 4111-1111-1111-1111."
]


In [ ]:
results = []

for idx, line in enumerate(test_texts, 1):
    try:
        output = redactor(text=line)
        results.append({
            "Input": line,
            "Redacted": output.redacted_text,
            "Entities": output.pii_entities
        })
    except Exception as e:
        results.append({
            "Input": line,
            "Redacted": "ERROR",
            "Entities": str(e)
        })


In [ ]:
import pandas as pd
import IPython.display as display

df = pd.DataFrame(results)
display.display(df)

,Input,Redacted,Entities
0,My name is Alice. Email: alice@example.com. SS...,My name is [REDACTED]. Email: [REDACTED]. SSN:...,"{""Name"": ""Alice"", ""Email"": ""alice@example.com""..."
1,Contact John via john.doe@gmail.com. He lives ...,Contact [REDACTED] via [REDACTED]. He lives at...,"{""Name"": ""John"", ""Email"": ""john.doe@gmail.com""..."
2,Sarah's phone number is 555-123-4567 and her a...,Sarah's phone number is [REDACTED] and her add...,"{""PHONE_NUMBER"": ""555-123-4567"", ""ADDRESS"": ""5..."
3,James works at Google. His email is james@goog...,James works at Google. His email is [REDACTED].,"{""email"": ""james@google.com""}"
4,"Emma lives in San Francisco, California. Her n...","Emma lives in San Francisco, California. Her n...","{""phone_number"": ""415-555-7890""}"
5,Liam's passport number is X12345678.,Liam's passport number is [REDACTED].,"{""X12345678"": ""Passport Number""}"
6,"Olivia’s home is 98 Hill Rd, Boston. Reach her...","[REDACTED]'s home is [REDACTED], Boston. Reach...","{""Name"": ""Olivia"", ""Address"": ""98 Hill Rd, Bos..."
7,William was born on 01/15/1990 and his SSN is ...,William was born on [REDACTED] and his SSN is ...,"{""DOB"": ""01/15/1990"", ""SSN"": ""321-54-9876""}"
8,Ava’s insurance policy number is INSU123456789.,Ava’s insurance policy number is [REDACTED].,"{""INSU123456789"": ""Insurance Policy Number""}"
9,Mason’s IP address is 192.168.1.100.,Mason’s IP address is [REDACTED].,"{""IP address"": [""192.168.1.100""]}"


## ✅ Lab 1: Things You Can Try

As you explore each section of this lab, here are a few ideas and challenges to help you deepen your learning and stretch your prompt engineering.

---

### 1. CVE Extraction from Noisy Logs

**Try This:**
- Modify the prompt to return **CVE IDs with context** (e.g., surrounding log lines)
- Ask the LLM to **group CVEs by year**, or tag them with **risk levels**
- Try feeding logs in **different formats** (CSV, JSON lines, etc.)
- Introduce noise or obfuscation in the log — does the LLM still work?

---

### 2. Secure Code Review with GenAI

**Try This:**
- Change the code snippet to include **multiple issues** (e.g., hardcoded secrets + SQL injection)
- Ask for **line numbers** where vulnerabilities occur
- Change the output format to **YAML or Markdown** (to simulate report generation)
- Add a prompt variant that asks the model to **write unit tests** for the fixed code
- Try removing part of the prompt — how does it affect the model's precision?

---

### 3. PII Redaction with spaCy + Regex

**Try This:**
- Extend regex patterns to cover:
  - IPv4 / IPv6 addresses
  - Credit card numbers
  - API keys or tokens
- Try **visualizing** the redacted vs original text side-by-side
- Challenge: Introduce **edge cases** like misformatted phone numbers or emails
- Try integrating this into a **streaming input (chat-style)**

---

### 4. PII Redaction with DSPy

**Try This:**
- Modify the DSPy signature to also return **reasoning steps** per PII detected
- Add a field for **PII type confidence score**
- Use DSPy with a **different LLM backend** (e.g., change `lm="openai/gpt-4"`)
- Try combining **DSPy with Guardrails** to enforce JSON validation

---

> 💡 Bonus Challenge: Combine what you’ve learned across labs — e.g., extract CVEs from logs that include PII, redact it first, then run analysis.





------------------------------------------------

## 🔁 Alternate Setup: Run with Ollama or OpenAI

If you'd like to rerun this notebook after the workshop, either using OpenAI or a free local model — here’s how to configure your environment.

---

### 🔹 Option 1: Use OpenAI GPT (3.5 or 4)

1. **Get your API Key:**  
   👉 [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys)

2. **Install the OpenAI SDK:**
   ```bash
   pip install openai


### 🔹 Option 2: Use Free Local LLMs with Ollama (e.g., LLaMA3)

You can run LLMs like LLaMA3 or Mistral locally on your machine using [Ollama](https://ollama.com/), which is free and privacy-friendly.

**Steps to use Ollama:**
1. Download and install Ollama → [https://ollama.com/download](https://ollama.com/download)
2. Run a model like `llama3` locally on your system
3. Send prompts to the local model using Python or curl

Ollama allows you to experiment with powerful open-source LLMs without needing cloud access or API keys — great for offline use, local testing, or privacy-sensitive projects.

> 💡 Ideal for developers who want to avoid API costs or test LLM behavior in sandboxed environments.


## Key Takeaways
1. **Structure matters more than length** in prompts
2. **Always validate LLM outputs** in security contexts  
3. **Hybrid approaches** (rules + AI) often work best
4. **Test edge cases** extensively before production use

##  Mapping to OWASP LLM Top 10

| Lab Task                            | OWASP LLM Risk                        | Description                                               |
|------------------------------------|--------------------------------------|-----------------------------------------------------------|
| CVE Extraction from logs           | **LLM01: Prompt Injection**          | Teaches prompt clarity to avoid misbehavior               |
| Secure Code Review                 | **LLM06: Insecure Output Handling**  | Ensures model output is structured, filtered, validated   |
| PII Redaction with spaCy/DSPy      | **LLM05: Sensitive Info Disclosure** | Redacts name, email, SSN from user/system input           |
| Weak prompt (before cleanup)       | **LLM04: Overreliance**              | Highlights model brittleness when prompts are vague       |
| DSPy use for redaction             | **LLM03: Training Data Poisoning**   | Begins the discussion on controlling output formats       |


## Aligning NIST AI Risk Management Framework (RMF)

| RMF Core Function | Lab Alignment                                             |
|-------------------|----------------------------------------------------------|
| **Map**           | Identify system logs, PII, and code inputs as use cases  |
| **Measure**       | Evaluate prompt performance, output quality              |
| **Manage**        | Use structured output and validations (e.g., JSON)       |
| **Govern**        | Discuss hallucinations, bias, sensitive data exposure    |

